## Implementing LSH from scratch

In recent years, with the explosive growth of data, there has been an increasing need to find efficient ways to search and retrieve similar items from large datasets. One popular method for solving this problem is locality-sensitive hashing (LSH), which has been widely used in various applications, such as image and video search, recommendation systems, and data de-duplication. 

In this session, we will explore the theory behind LSH and implement it from scratch using Python. We will also apply LSH to a real-world dataset and evaluate its performance. By the end of this session, we will have a solid understanding of LSH and its practical applications, as well as the ability to implement LSH from scratch using Python.

Let's get started by importing some libraries and data file.

In [14]:
# Mounting google drive as we are loading the data from google drive
# To do this we need to import "drive" from "google.colab"
from google.colab import drive
drive.mount("gdrive/", force_remount=True)

Mounted at gdrive/


In [15]:
# Loading data from csv file
import pandas as pd

# Assigning the path of the file to variable "data_path"
data_path = "gdrive/My Drive/My DB/"

# Reading the CSV file by using Pandas "read_csv" function and convert it into a pandas DataFrame object. 
df = pd.read_csv(data_path + "lsh_assignment_data.csv")

# Printing the dataframe "df"
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...


We can see that our Dataframe "df" consist of 2225 rows and 2 columns- "category" and "text". "text" consists of a group of text in some topics. While "category" consist of the topics corresponding to the column "text". 

**Data overview:**

We are going to use "value_count()" method from pandas library which is used to count the unique values in a column of a DataFrame. It returns a Series containing the counts of unique values. The resulting Series is in descending order so that the most frequently-occurring values appear first.

In [16]:
# Data overview
df["category"].value_counts()

sport            509
business         508
politics         415
tech             399
entertainment    384
Name: category, dtype: int64

**Creating Train and Test Datasets:**

Creating training and test data is an important step in Machine learning. This allows us to evaluate the model's performance on unseen data to ensure that it can generalize well to new, unseen data points.

The training set is used to train the model, while the test set is used to evaluate the model's performance on new, unseen data. The test set is kept separate from the training set during the training phase to avoid overfitting. Overfitting occurs when the model becomes too complex and starts to learn the noise and idiosyncrasies of the training data, rather than the underlying patterns and relationships. As a result, the model's performance on the training data may be very good, but its performance on new, unseen data will be poor.

By evaluating the model on a separate test set, we can get an estimate of how well the model will perform on new, unseen data. This helps us to choose the best model and tune its hyperparameters to achieve the best performance.

Here in our dataset, the last 10 datapoints are unlabelled so we assigned them to "test_data" and the datapoints into "train_data". Our machine model will identify the labels of the datapoints in the "test_data".

In [17]:
# Assigning training and test data
train_data = df.iloc[:-10]
test_data = df.iloc[-10:]

We can look into our training and test data by printing them

In [18]:
# For train_data with labels in the column named "category".
train_data

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2210,politics,teens know little of politics teenagers ques...
2211,entertainment,lopez misses uk charity premiere jennifer lope...
2212,business,christmas shoppers flock to tills shops all ov...
2213,tech,progress on new internet domains by early 2005...


In [19]:
# test_data without labels 
test_data

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


#Custom  Implementation

We are importing some more libraries which will be useful in the course of our modelling process.

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from collections import defaultdict
from numpy import linalg as la

TfidfVectorizer is a valuable tool in natural language processing and machine learning. It stands for Term Frequency-Inverse Document Frequency, which is a statistical measure that reflects how important a word is to a document in a collection or corpus. It helps in text classification and clustering by transforming text into a numerical representation, improving the efficiency and effectiveness of algorithms. It can also aid in text preprocessing by removing stop words, stemming, and converting text to lowercase.

NumPy is a powerful Python library for scientific computing that enables efficient numerical operations on large data sets. It provides a multi-dimensional array object, tools for working with these arrays, and mathematical functions to operate on them.


In [21]:
def predictLabels (test_data):
    # Generating vectorized train_data using sklearns built in tfidf vectorizer
    tfidf = TfidfVectorizer(ngram_range = (2,3), max_features = 4000, min_df = 10)    
    # Fitting the vectorizer to the data and transforming the data into a document-term matrix with TF-IDF weights. 
    train_vectors = tfidf.fit_transform(train_data.text)

    # Assigning number of nearest neighbours
    k = 11

    # Assigning number of hyperplanes
    num_hyperplanes = 5                                                                     
    # Setting random seed of the NumPy random number generator to 0, 
    # it to ensure that the same sequence of random numbers is generated every time the code is run.
    np.random.seed(0)
    # Generating 5 normally distributed random hyperplanes with mean=0 & variance=1
    hyperplanes = np.random.normal(0, 1, (num_hyperplanes, train_vectors.shape[1]))

    # Creating an array of integers in a reverse order starting from num_hyperplanes-1 and ending at 0 (inclusive). 
    # The step parameter is set to -1, which means that each subsequent number in the array is reduced by 1. 
    a = np.arange(num_hyperplanes-1, -1, step = -1)                             
    # Generating power of two, 1<<a = 1*2^a
    power_of_two = 1 << a     

    # Calculating distances of training datapoints from hyperplanes by calling the function "find_dist()"                                          
    score = find_dist(train_vectors, hyperplanes, power_of_two)  
    
    # Initializing dictionary with list of values               
    table = defaultdict(list)                                                   
    # Creating the dictionary while placing the points into their respective bins
    for i in range(len(score)):                                                 
        table[score[i]].append(i)

    # Vectorizing the text in test data with TfidfVectorizer
    Xq = tfidf.transform(test_data.text)  
    # Calculating distances of query points from hyperplanes                                      
    Q_score = list(find_dist(Xq, hyperplanes, power_of_two))                    

    predicted_labels = []      
    
    # Finding predicted labels for the query points by looping through each datapoint                                                
    for i in range(len(Q_score)):                                               
        locality = []                                                          
        locality = np.array(table[Q_score[i]])
        cos_sim = []
        # Finding cosine similarities of all the datapoints in the locality
        for j in locality:                                                      
            cosine = (train_vectors[j].dot(Xq[i].T)).todense().item() / (la.norm(train_vectors[j].toarray()) * (la.norm(Xq[i].toarray())))
            cos_sim.append(cosine)
        
        # Sorting the similarities indices in descending order
        # The closer the cosine similarity value is to 1, the more similar the vectors are considered to be. 
        argsorted_cos_sim = np.argsort(cos_sim)[::-1]  
        # Considering only the nearest k=11 neighbors                         
        neighbors = locality[argsorted_cos_sim[:k]] 

        # Predicting possible query points labels by using training data's labels                            
        predictions = list(train_data.category[neighbors])    
        # Getting the label by maximum frequented possible labels                  
        label = max(set(predictions), key = predictions.count)  
        # Forming a list of labels of all the query points                
        predicted_labels.append(label)  
          
    # Returning predicted_labels to the main function                                      
    return(predicted_labels)                                                    

def find_dist(vector, hyperplanes, power_of_two):  
    #generating binary bits 0 for the points on the -ve side and 1 for +ve of the hyperplanes                             
    bin_bits = vector.dot(hyperplanes.T) <= 0  
    #Converting the binary bits into corresponding decimal values                                                                                           
    dec_val = bin_bits.dot(power_of_two)                                        
    return dec_val

## GRADER CELL

Grader cells are designed to check whether the above codes are successfully executed or not. 

This cell will print "Success" if the accuracy of the implementation of the predictLabels() function is 80% or more. Else, it will print "Failed

In [37]:
# Importing numpy as np
import numpy as np

# Predict labels using the predictLabels() function
Y_custom = np.array(predictLabels(test_data))

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

# Calculating accuracy by comparing Y_grader and Y_custom
accuracy = np.sum(Y_grader == Y_custom) * 10
# Compare Y_grader and Y_custom
if accuracy>=80:
  print("******* Success *******","\n\nAccuracy Achieved:",accuracy,'%')
else:
  print("####### Failed #######")
  print("\n","."*70)
  print("\Y_grader = \n\n", Y_grader)
  print("\n","."*70)
  print("\Y_custom = \n\n", Y_custom)


******* Success ******* 

Accuracy Achieved: 90 %
